This step is taken after the occurrence.txt file has been extracted from the dwca zip file, and saved as a reduced CSV file.

This script will load the occurrence csv file, split it by "acceptedTaxonKey" and save each as a separate CSV file

In [ ]:
import os
import pandas as pd
import numpy as np
import concurrent.futures
import logging

In [ ]:
home_dir = os.path.dirname(os.getcwd())

data_dir = "/Users/lbokeria/Documents/projects/gbif-species-trainer-data/"
# data_dir = "/bask/projects/v/vjgo8416-amber/data/gbif-species-trainer-AMI-fork/"

save_folder = "occurrence_dataframes"

save_dir = os.path.join(data_dir,save_folder)

# If save_dir doesn't exist, create it 
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

filename = "Sesiidae"

occ_df_path = os.path.join(data_dir,"dwca_files","occurrence_"+filename+".csv")

In [ ]:
kwargs = {}

kwargs['parse_dates'] = True
kwargs['on_bad_lines'] = "skip"

occ_df = pd.read_csv(occ_df_path, **kwargs)

In [ ]:
# Select only numeric acceptedTaxonKey rows
def is_number(x):
    try:
        # Check for NaN
        if pd.isna(x):
            return False
        # Try converting the element to a float.
        float(x)  
        return True  # If conversion is successful, it's a number or a number string.
    except ValueError:  # If conversion fails, it's not a number string.
        return False
    except TypeError:  # If type conversion is not possible (e.g., for NaNs), also not considered as a number.
        return False

In [ ]:
mask = occ_df["acceptedTaxonKey"].apply(is_number)

occ_df_preprocessed = occ_df[mask]

In [ ]:
# Try saving as dataframes
groups = occ_df_preprocessed.groupby("acceptedTaxonKey")

In [ ]:
groups.ngroups

In [ ]:
def save_group(group):
    
    group_name, group_df = group
    filename = f"{group_name}.csv"
    
    try:
        group_df.to_csv(os.path.join(save_dir, filename), index=False)
        # print(f"Success for {group_name}")
    except:
        print(f"Couldn't save {group_name}")

groups = list(occ_df_preprocessed.groupby("acceptedTaxonKey"))

In [ ]:
# Use multi-threading
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(save_group, groups)

In [ ]:
for group_name, group_df in groups:
    filename = f'{group_name}.csv'
    
    try:
    
        group_df.to_csv(os.path.join(save_dir,filename), index=False)
        
    except:
        print(f"Couldn't save {group_name}")